In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging

# --- 프로젝트 루트 경로 설정 ---
# (주의: 이 스크립트가 'notebooks' 폴더에 있다고 가정)
try:
    # 이 스크립트 파일(notebooks/...)의 상위 폴더(notebooks)의 상위 폴더(프로젝트 루트)
    PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
except NameError:
    # __file__ 변수가 없는 환경 (e.g., REPL)을 위한 대체
    PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    print(f"Added project root to path: {PROJECT_ROOT}")

# --- src 모듈 임포트 ---
try:
    from src.utils import load_config
    from src.data_loader import load_dataset
except ImportError as e:
    print(f"Error importing src modules: {e}")
    print("Please ensure 'src' folder exists in the project root and sys.path is correct.")
    # sys.exit(1) # 노트북에서는 종료 대신 경고만 출력

# 로깅 기본 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(module)s - %(message)s')
logger = logging.getLogger(__name__)

print("Libraries and modules imported successfully.")

In [ ]:
# --- 2. 설정 파일 로드 (`config.yaml`) ---
print("\n--- 2. Loading Config File ---")
CONFIG_PATH = os.path.join(PROJECT_ROOT, "configs", "config.yaml")
config = None
try:
    config = load_config(CONFIG_PATH)
    print("Config file loaded successfully.")
    print("--- Config Content (Partial) ---")
    print(f"Default T_max: {config.get('default_t_max')}")
    print(f"Models defined: {[m.get('name') for m in config.get('models', [])]}")
    print(f"Data paths: {list(config.get('data_paths', {}).keys())}")
except Exception as e:
    logger.error(f"Config 파일 로드 실패 ({CONFIG_PATH}): {e}")

In [ ]:
# --- 3. 벤치마크 데이터셋 로드 및 탐색 ---
print("\n--- 3. Loading and Exploring Datasets ---")
if config:
    dataset_paths = config.get('data_paths', {})
    if not dataset_paths:
        logger.warning("config.yaml에 'data_paths'가 정의되지 않았거나 비어있습니다.")
    else:
        # 로드된 데이터를 저장할 딕셔너리
        loaded_datasets = {}
        
        for dataset_key, relative_path in dataset_paths.items():
            print(f"\n{'='*30}")
            print(f"Loading dataset: '{dataset_key}'")
            print(f"Relative path: {relative_path}")
            
            dataset_abs_path = os.path.join(PROJECT_ROOT, relative_path)
            
            try:
                # 데이터 로더 함수 호출
                data = load_dataset(dataset_name=dataset_key, file_path=dataset_abs_path)
                
                if data:
                    loaded_datasets[dataset_key] = data
                    print(f"[성공] '{dataset_key}' 로드 완료. 총 {len(data)}개 샘플.")
                    
                    # 1. 키 구조 확인
                    print(f"  - 첫 번째 샘플의 키: {list(data[0].keys())}")
                    
                    # 2. 'query' 또는 'question' 키 존재 확인
                    query_key = 'query' if 'query' in data[0] else ('question' if 'question' in data[0] else None)
                    if query_key:
                        print(f"  - [확인] 입력 키('{query_key}')가 존재합니다.")
                    else:
                        logger.error(f"  - [오류!] '{dataset_key}'에 'query' 또는 'question' 키가 없습니다. data_loader.py 수정 필요.")
                    
                    # 3. 데이터프레임으로 변환하여 샘플 확인
                    df = pd.DataFrame(data)
                    print("  - 랜덤 샘플 5개:")
                    # display()는 Jupyter/IPython 환경에서 표를 예쁘게 보여줌
                    display(df.sample(min(5, len(df))))
                    
                    # 4. 쿼리 길이 분석
                    if query_key:
                        df['query_length'] = df[query_key].astype(str).apply(len)
                        print("  - 쿼리 길이 통계:")
                        display(df['query_length'].describe())
                        
                        # 5. 쿼리 길이 시각화
                        plt.figure(figsize=(10, 4))
                        sns.histplot(df['query_length'], kde=True, bins=50)
                        plt.title(f"'{dataset_key}' 쿼리 길이 분포")
                        plt.xlabel("Query Length (characters)")
                        plt.ylabel("Count")
                        print(f"  - '{dataset_key}'의 쿼리 길이 분포 그래프를 생성합니다.")
                        plt.show()
                        
                else:
                    logger.warning(f"'{dataset_key}' 로드 결과가 비어있습니다.")
                    
            except FileNotFoundError:
                logger.error(f"[오류!] '{dataset_key}' 파일을 찾을 수 없습니다: {dataset_abs_path}")
            except ValueError as e:
                logger.error(f"[오류!] '{dataset_key}' 파일 형식 오류 또는 처리 불가: {e}")
            except Exception as e:
                logger.error(f"[오류!] '{dataset_key}' 로딩 중 알 수 없는 오류 발생: {e}", exc_info=True)
else:
    logger.error("Config 파일이 로드되지 않아 데이터 탐색을 진행할 수 없습니다.")

print("\n--- 4. 탐색 완료 ---")